In [8]:
import numpy as np
from PIL import Image
from stl import mesh
from PIL import Image

In [24]:
def stl_to_birdseye(stl_file, resolution, output_file, boundary, height_window):
    # Load the STL file and create a mesh object
    mesh_data = mesh.Mesh.from_file(stl_file)

    # Convert the mesh vertices to a point cloud
    points = mesh_data.vectors.reshape(-1, 3)

    # Cut off points outside a certain rectangle
    points = points[(points[:, 0] > boundary[0]) & (points[:, 0] < boundary[1])]
    points = points[(points[:, 1] > boundary[2]) & (points[:, 1] < boundary[3])]

    # Cut off points outside a certain height window
    points = points[(points[:, 2] > height_window[0]) & (points[:, 2] < height_window[1])]
    
    # Transform to bird's-eye view
    bev_points = points[:, :2]  # Remove Z-axis information

    # Define the grid parameters
    min_x = np.min(bev_points[:, 0])
    min_y = np.min(bev_points[:, 1])
    max_x = np.max(bev_points[:, 0])
    max_y = np.max(bev_points[:, 1])
    range_x = max_x - min_x
    range_y = max_y - min_y

    # Calculate the grid size based on the range of coordinates
    grid_size_x = int(range_x / resolution) + 1
    grid_size_y = int(range_y / resolution) + 1

    # Map the points to the grid with intensity mapping
    bev_grid = np.zeros((grid_size_y, grid_size_x))

    for point in bev_points:
        x = int((point[0] - min_x) / range_x * (grid_size_x - 1))
        y = int((point[1] - min_y) / range_y * (grid_size_y - 1))
        bev_grid[y, x] = 1  # Set intensity to 1 when a point exists

    # Generate the bird's-eye view image
    image = Image.fromarray((bev_grid * 255).astype(np.uint8))

    # Mirror the image vertically
    mirrored_image = image.transpose(Image.FLIP_TOP_BOTTOM)

    # Save the mirrored image as PNG
    mirrored_image.save(output_file)

    # Get coordinates of the (x=0, y=0) point in the bird's-eye view image
    x_bev = int((0 - min_x) / range_x * (grid_size_x - 1))
    y_bev = int((0 - min_y) / range_y * (grid_size_y - 1))
    pixel_x = -x_bev*resolution
    pixel_y = -y_bev*resolution
    print(f"In cost_map.yaml, set: \norigin: [{pixel_x}, {pixel_y}, 0.0]")


In [25]:
# Input stl and output png file
stl_file = "stl/05_24_test.stl"
output_file = "bev_images/5_24_test.png"
# Resolution of the grid in meters
resolution = 0.05
# Cut off boundary and height window
boundary = [-100, 100, -100, 100]
height_window = [0.1, 1.0]

# Call the function and save the image
stl_to_birdseye(stl_file, resolution, output_file, boundary, height_window)

In cost_map.yaml, set: 
origin: [-23.8, -7.5, 0.0]
